In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

Link And Header

In [ ]:
URL="https://www.amazon.in/s?k=laptops&crid=AF964DR5SHCU&sprefix=laptops%2Caps%2C225&ref=nb_sb_noss_2"

HEADERS= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36",'Accept-Language': 'en-US, en;q=0.5'
}

Home page


In [ ]:
webpage=requests.get(URL,headers=HEADERS)

#parses the html content
soup = BeautifulSoup(webpage.content,"html.parser") #this will convert into html file

Product page

In [ ]:
#through css
product_container = soup.select("a.a-link-normal.s-line-clamp-2.s-link-style.a-text-normal")


Product Info

In [ ]:
with open("amazon_laptops.csv", mode="w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)

    writer.writerow(["Title", "Price", "Discount", "MRP", "Availability", "Rating"])

    for page_num in range(1,20):
        URL_page=URL.format(page_num)
        webpage=requests.get(URL_page,headers=HEADERS)
        soup = BeautifulSoup(webpage.content,"html.parser")

        product_container = soup.select("a.a-link-normal.s-line-clamp-2.s-link-style.a-text-normal")


        for product in product_container:
            get_link=product.get('href')
            product_link='https://www.amazon.in'+get_link

            #Delay
            time.sleep(random.uniform(1, 3))

            product_webpage=requests.get(product_link, headers=HEADERS)
            new_soup=BeautifulSoup(product_webpage.content, 'html.parser')

            #title
            try:    
                title_text=new_soup.find("span", attrs={'id':"productTitle"}).text.strip()
            except AttributeError:
                title_text="NA"

            #price
            try:

                symbol=new_soup.find("span", attrs={'class':"a-price-symbol"}).text.strip()
                price=new_soup.find("span", attrs={"class":"a-price-whole"}).text.strip()
                discounted_price_text=symbol+price
            except AttributeError:
                discounted_price_text="NA"

            #Discount
            try:
                discount=new_soup.find("span", attrs={'class':"a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage"}).text.strip()
            except AttributeError:
                discount="NA"

            #MRP
            try:
                Original_amount=new_soup.find("span", attrs={'class':"a-price a-text-price"}).find("span", attrs={'class':"a-offscreen"}).text.strip()
            except AttributeError:
                Original_amount="NA"

            #Availabilty 
            try:
                available=new_soup.find("span", attrs={"class":"a-size-medium a-color-success"}).text.strip()
            except AttributeError:
                available="NA"

            #Rating
            try:
                rating=new_soup.find("span", attrs={'class':'a-icon-alt'}).text.strip()
            except AttributeError:
                rating="NA"

            writer.writerow([title_text, discounted_price_text, discount, Original_amount, available, rating])

            print(f"Saved: {title_text}")

    
    

    